In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q ta tqdm

***FOR STOCKS***

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import os, glob
from tqdm.notebook import tqdm
from ta.trend import EMAIndicator, MACD
from ta.volume import OnBalanceVolumeIndicator
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from pytz import timezone
from datetime import datetime
from sklearn.metrics import mean_squared_error, r2_score

# 🚀 Initialize Spark (optional, for distributed future processing)
try:
    spark = SparkSession.builder.master("local[*]").appName("StockReturns").getOrCreate()
except Exception as e:
    print(f"Could not initialize Spark Session: {e}. Proceeding without Spark.")
    spark = None

# 🔧 Parameters
DATA_END_DATE = pd.to_datetime("2025-06-18").tz_localize("Asia/Kolkata")
TODAY = DATA_END_DATE

TOP_N = 10
MIN_YEARS_REQUIRED = 3
PREDICTION_HORIZON_YEARS = 1
DAYS_FORWARD = 252 * PREDICTION_HORIZON_YEARS

MIN_OBS_INITIAL = 252 * MIN_YEARS_REQUIRED
MIN_OBS_FOR_MODEL = 252 * 2.5

DATA_PATH = r"/content/drive/MyDrive/nse_stock_data/*.csv"
OUTPUT_CSV_PATH = "stock_predictions_xgboost_r2_composite_risk.csv"

files = glob.glob(DATA_PATH)

results, failures = [], []
preprocessing_failures = []

print(f"🔍 Total files: {len(files)}")

# --- Helper Functions ---

def assign_volatility_risk_band(annualized_volatility_pct):
    if pd.isna(annualized_volatility_pct):
        return "Unknown"
    elif annualized_volatility_pct < 15:
        return "Low"
    elif annualized_volatility_pct < 30:
        return "Medium"
    else:
        return "High"

def calculate_composite_risk_score(vol_pct, pe_ratio, beta):
    if pd.isna(vol_pct) or pd.isna(pe_ratio) or pd.isna(beta):
        return np.nan

    norm_vol = min(1, max(0, (vol_pct - 10) / 40))
    norm_pe = min(1, max(0, (pe_ratio - 15) / 35))
    if pe_ratio < 10: norm_pe = 0.5

    norm_beta = min(1, max(0, (beta - 0.5) / 1.5))

    weight_vol = 0.40
    weight_pe = 0.30
    weight_beta = 0.30

    composite_score = (weight_vol * norm_vol) + (weight_pe * norm_pe) + (weight_beta * norm_beta)
    return composite_score

def assign_composite_risk_band(composite_score):
    if pd.isna(composite_score): return "Unknown"
    if composite_score < 0.3:
        return "Low"
    elif composite_score < 0.6:
        return "Medium"
    else:
        return "High"

def generate_features(df_input):
    df = df_input.copy()

    cols_to_numeric = ['Open', 'High', 'Low', 'Close', 'Volume']
    if 'P/E Ratio' in df.columns: cols_to_numeric.append('P/E Ratio')
    if 'Beta' in df.columns: cols_to_numeric.append('Beta')

    for col in cols_to_numeric:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    required_cols_for_ta_indicators = ['Close', 'Volume']
    ohlc_cols = ['High', 'Low', 'Close']

    if not all(col in df.columns and not df[col].isnull().all() for col in required_cols_for_ta_indicators):
        return None

    df['Return_1d'] = df['Close'].pct_change(1, fill_method=None)

    df['Return_1w'] = df['Close'].pct_change(5, fill_method=None)
    df['Return_1m'] = df['Close'].pct_change(21, fill_method=None)
    df['Return_6m'] = df['Close'].pct_change(126, fill_method=None)
    df['Volatility_21d'] = df['Close'].rolling(21).std()

    if not df['Return_1d'].isnull().all():
        df['Annualized_Volatility'] = df['Return_1d'].rolling(window=252).std() * np.sqrt(252)
    else:
        df['Annualized_Volatility'] = np.nan

    has_ohlc_for_indicators = all(col in df.columns and not df[col].isnull().all() for col in ohlc_cols)

    if not df['Close'].isnull().all():
        df['EMA_20'] = EMAIndicator(df['Close'], window=20).ema_indicator()
        df['MACD'] = MACD(df['Close']).macd()
    else:
        df['EMA_20'] = np.nan
        df['MACD'] = np.nan

    if has_ohlc_for_indicators:
        df['ADX_14'] = ADXIndicator(df['High'], df['Low'], df['Close'], window=14).adx()
    else:
        df['ADX_14'] = np.nan

    if not df['Close'].isnull().all() and not df['Volume'].isnull().all():
        df['OBV'] = OnBalanceVolumeIndicator(df['Close'], df['Volume']).on_balance_volume()
    else:
        df['OBV'] = np.nan

    if not df['Close'].isnull().all():
        df['RSI_14'] = RSIIndicator(df['Close'], window=14).rsi()
    else:
        df['RSI_14'] = np.nan

    if has_ohlc_for_indicators:
        df['Stoch_K'] = StochasticOscillator(df['High'], df['Low'], df['Close'], window=14).stoch()
    else:
        df['Stoch_K'] = np.nan

    if not df['Close'].isnull().all():
        bb = BollingerBands(df['Close'], window=20)
        df['BB_width'] = bb.bollinger_wband()
    else:
        df['BB_width'] = np.nan

    df['Momentum_21'] = df['Close'] - df['Close'].shift(21)
    df['Vol_Change_1w'] = df['Volume'].pct_change(5)

    return df

# --- Preprocessing Stage Function ---
def preprocess_stock_data(file_path, DATA_END_DATE, MIN_OBS_INITIAL, MIN_OBS_FOR_MODEL, DAYS_FORWARD):
    stock_symbol = os.path.basename(file_path).split("_")[0]

    try:
        df_full = pd.read_csv(file_path, parse_dates=['Date'])
        df_full = df_full.sort_values("Date").reset_index(drop=True)

        df = df_full[df_full['Date'] <= DATA_END_DATE].copy()

        if df.shape[0] < MIN_OBS_INITIAL:
            return None, None, f"Insufficient initial data after date filtering ({df.shape[0]} rows, {MIN_OBS_INITIAL} required) up to {DATA_END_DATE.date()}."

        required_fundamental_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        cols_for_numeric_conversion = required_fundamental_cols.copy()
        if 'P/E Ratio' in df.columns: cols_for_numeric_conversion.append('P/E Ratio')
        if 'Beta' in df.columns: cols_for_numeric_conversion.append('Beta')

        for col in cols_for_numeric_conversion:
            if col not in df.columns:
                if col in required_fundamental_cols:
                    return None, None, f"Missing essential column: '{col}'."
                else:
                    df[col] = np.nan
            else:
                df[col] = pd.to_numeric(df[col], errors='coerce')
                if col in required_fundamental_cols and df[col].isnull().all():
                    return None, None, f"Essential column '{col}' is entirely NaN after numeric conversion."

        df.dropna(subset=required_fundamental_cols, inplace=True)
        if df.empty:
            return None, None, "DataFrame became empty after dropping NaNs from essential columns."

        if df.shape[0] < MIN_OBS_INITIAL:
            return None, None, f"Insufficient data after dropping NaNs from essential columns ({df.shape[0]} rows, {MIN_OBS_INITIAL} required)."

        df_features = generate_features(df)
        if df_features is None:
            return None, None, "Failed to generate features due to missing essential data for indicator calculation."

        df_features['Target_Return'] = df_features['Close'].shift(-DAYS_FORWARD) / df_features['Close'] - 1

        if df_features['Volume'].mean() < 10000:
            return None, None, f"Low average volume ({df_features['Volume'].mean():.0f} < 10000)."

        features_for_model = ['Return_1w', 'Return_1m', 'Return_6m', 'Volatility_21d',
                              'EMA_20', 'MACD', 'OBV', 'RSI_14', 'BB_width',
                              'Momentum_21', 'Vol_Change_1w', 'ADX_14', 'Stoch_K',
                              'Annualized_Volatility']

        for feature in features_for_model:
            if feature not in df_features.columns:
                df_features[feature] = np.nan
            df_features[feature] = pd.to_numeric(df_features[feature], errors='coerce')

        df_model_ready = df_features[features_for_model + ['Target_Return']].replace([np.inf, -np.inf], np.nan).dropna()

        if df_model_ready.shape[0] < MIN_OBS_FOR_MODEL:
            return None, None, f"Insufficient observations for model training after feature generation and NaN drop ({df_model_ready.shape[0]} < {MIN_OBS_FOR_MODEL} required)."

        return df_model_ready, df_features, stock_symbol

    except Exception as e:
        return None, None, str(e)


# --- Main Processing Loop ---
print("Starting stock analysis (XGBoost Only)...")
for file in tqdm(files, desc="⏳ Processing files"):
    stock_symbol_current = os.path.basename(file).split("_")[0]

    # --- Preprocessing Stage ---
    processed_data_for_model, df_for_latest_features, preprocessing_error_msg = \
        preprocess_stock_data(file, DATA_END_DATE, MIN_OBS_INITIAL, MIN_OBS_FOR_MODEL, DAYS_FORWARD)

    if processed_data_for_model is None:
        preprocessing_failures.append((stock_symbol_current, preprocessing_error_msg))
        continue

    # Extract X_all and y_all from the successfully processed data
    features_for_model = ['Return_1w', 'Return_1m', 'Return_6m', 'Volatility_21d',
                          'EMA_20', 'MACD', 'OBV', 'RSI_14', 'BB_width',
                          'Momentum_21', 'Vol_Change_1w', 'ADX_14', 'Stoch_K',
                          'Annualized_Volatility']

    X_all = processed_data_for_model[features_for_model]
    y_all = processed_data_for_model['Target_Return'] * 100

    # --- Model Training Stage (XGBoost Only) ---
    try:
        if X_all.shape[0] < 5:
            raise ValueError(f"Not enough data points ({X_all.shape[0]}) for XGBoost train-test split (min 5).")

        X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X_all, y_all, shuffle=False, test_size=0.2)

        if X_train_xgb.empty or y_train_xgb.empty:
            raise ValueError("XGBoost Training set is empty after train-test split.")

        xgb_model = XGBRegressor(
            n_estimators=300, max_depth=6, learning_rate=0.1,
            subsample=0.9, colsample_bytree=0.9, random_state=42, n_jobs=-1
        )
        xgb_model.fit(X_train_xgb, y_train_xgb)

        # --- Handle Error Group C: Predict from last available date ---
        latest_features_df_raw = df_for_latest_features[df_for_latest_features['Date'] <= DATA_END_DATE].tail(1)

        if latest_features_df_raw.empty:
            raise ValueError(f"No data available on or before prediction date {DATA_END_DATE.date()} for final feature extraction (XGBoost results).")

        # --- Handle Error Group B: Impute NaNs in latest_features_xgb with median from X_train_xgb ---
        latest_features_xgb = latest_features_df_raw[features_for_model].replace([np.inf, -np.inf], np.nan)

        cols_with_nan = latest_features_xgb.columns[latest_features_xgb.isnull().any()].tolist()

        for col in cols_with_nan:
            if col in X_train_xgb.columns:
                median_val = X_train_xgb[col].median()
                latest_features_xgb[col] = latest_features_xgb[col].fillna(median_val)
            else:
                latest_features_xgb[col] = latest_features_xgb[col].fillna(0) # Fallback

        if latest_features_xgb.isnull().any().any() or latest_features_xgb.empty:
             raise ValueError(f"Latest features for XGBoost prediction (on {DATA_END_DATE.date()}) are incomplete or contain unhandled NaNs after imputation.")

        pred_return_xgb = xgb_model.predict(latest_features_xgb)[0]
        cagr_xgb = ((pred_return_xgb / 100 + 1) ** (1 / PREDICTION_HORIZON_YEARS)) - 1
        cagr_pct_xgb = cagr_xgb * 100

        y_pred_xgb = xgb_model.predict(X_test_xgb)

        # Calculate R2 Score instead of RMSE and RMSE (%)
        r2_xgb = r2_score(y_test_xgb, y_pred_xgb)

        # --- Extract values for risk calculation ---
        actual_as_of_date = latest_features_df_raw['Date'].iloc[0]

        latest_pe_ratio = None
        if 'P/E Ratio' in latest_features_df_raw.columns:
            latest_pe_ratio = pd.to_numeric(latest_features_df_raw['P/E Ratio'].iloc[0], errors='coerce')

        latest_beta = None
        if 'Beta' in latest_features_df_raw.columns:
            latest_beta = pd.to_numeric(latest_features_df_raw['Beta'].iloc[0], errors='coerce')

        latest_annualized_volatility_val = None
        if 'Annualized_Volatility' in latest_features_xgb.columns and not latest_features_xgb.empty:
            latest_annualized_volatility_val = latest_features_xgb['Annualized_Volatility'].iloc[0] * 100

        composite_risk_score = calculate_composite_risk_score(
            vol_pct=latest_annualized_volatility_val,
            pe_ratio=latest_pe_ratio,
            beta=latest_beta
        )

        results.append({
            'Symbol': df_for_latest_features['Symbol'].iloc[0] if 'Symbol' in df_for_latest_features.columns else stock_symbol_current,
            'Company Name': df_for_latest_features['Company Name'].iloc[0] if 'Company Name' in df_for_latest_features.columns else "Unknown",
            'Prediction Horizon (Years)': PREDICTION_HORIZON_YEARS,
            'As Of Date': actual_as_of_date.strftime("%Y-%m-%d"),
            'Predicted Total Return (%)': round(float(pred_return_xgb), 2) if pd.notnull(pred_return_xgb) else None,
            'Predicted CAGR (%)': round(float(cagr_pct_xgb), 2) if pd.notnull(cagr_pct_xgb) else None,
            'Annualized Volatility (%)': round(float(latest_annualized_volatility_val), 2) if pd.notnull(latest_annualized_volatility_val) else None,
            'Latest P/E Ratio': round(float(latest_pe_ratio), 2) if pd.notnull(latest_pe_ratio) else None,
            'Latest Beta': round(float(latest_beta), 2) if pd.notnull(latest_beta) else None,
            'Composite Risk Score': round(float(composite_risk_score), 4) if pd.notnull(composite_risk_score) else None,
            'Risk Band (Composite)': assign_composite_risk_band(composite_risk_score),
            'Risk Band (Volatility Only)': assign_volatility_risk_band(latest_annualized_volatility_val),
            'Conservative Band': f"{round(float(cagr_pct_xgb - 1), 2)}% – {round(float(cagr_pct_xgb), 2)}%" if pd.notnull(cagr_pct_xgb) else "N/A",
            'Expected Band': f"{round(float(cagr_pct_xgb + 1), 2)}% – {round(float(cagr_pct_xgb + 3), 2)}%" if pd.notnull(cagr_pct_xgb) else "N/A",
            'Optimistic Band': f"{round(float(cagr_pct_xgb + 4), 2)}% – {round(float(cagr_pct_xgb + 8), 2)}%" if pd.notnull(cagr_pct_xgb) else "N/A",
        })

    except Exception as e:
        failures.append((stock_symbol_current, f"Model Stage Error (XGBoost): {str(e)}"))

# ✅ Final Output
results_df = pd.DataFrame(results)

# --- NEW: Filter out stocks with Predicted CAGR >= 50% ---
initial_processed_count = len(results_df)
results_df_filtered = results_df[results_df['Predicted CAGR (%)'] < 50].copy()
filtered_out_count = initial_processed_count - len(results_df_filtered)
if filtered_out_count > 0:
    print(f"\n🗑️ Filtered out {filtered_out_count} stocks with Predicted CAGR >= 50%.")
# --- END NEW FILTER ---


print(f"\n✅ Total Processed (after filtering): {len(results_df_filtered)} / {len(files)}")


if not results_df_filtered.empty: # Use filtered DataFrame here
    results_df_filtered.to_csv(OUTPUT_CSV_PATH, index=False)
    print(f"\n📄 All stock predictions saved to {OUTPUT_CSV_PATH}")
else:
    print("\n⚠️ No valid stock processed (or all filtered out) to save to CSV.")

if len(results_df_filtered): # Use filtered DataFrame here
    from IPython.display import display
    top_stocks_df = results_df_filtered.sort_values(by='Predicted Total Return (%)', ascending=False).head(TOP_N).reset_index(drop=True)
    display(top_stocks_df.style.set_caption(f"Top {TOP_N} Stocks with Predicted Returns and Confidence Bands (Horizon: {PREDICTION_HORIZON_YEARS} Year(s), As Of: {DATA_END_DATE.strftime('%Y-%m-%d')})"))
else:
    print("❌ No valid stock processed (or all filtered out) to display top stocks.")

# 🚨 Failed Files Log - Separated for clearer debugging
if preprocessing_failures:
    print("\n❌ Preprocessing Stage Failures (sample):")
    preprocessing_failures.sort(key=lambda x: x[1])
    for f, e in preprocessing_failures[:20]:
        print(f"{f} → {e}")
    if len(preprocessing_failures) > 20:
        print(f"... and {len(preprocessing_failures) - 20} more preprocessing failures (total {len(preprocessing_failures)})")

if failures: # Model stage failures
    print("\n❌ Model Stage Failures (sample):")
    failures.sort(key=lambda x: x[1])
    for f, e in failures[:20]:
        print(f"{f} → {e}")
    if len(failures) > 20:
        print(f"... and {len(failures) - 20} more model stage failures (total {len(failures)})")

🔍 Total files: 2092
Starting stock analysis (XGBoost Only)...


⏳ Processing files:   0%|          | 0/2092 [00:00<?, ?it/s]


🗑️ Filtered out 396 stocks with Predicted CAGR >= 50%.

✅ Total Processed (after filtering): 1057 / 2092

📄 All stock predictions saved to stock_predictions_xgboost_r2_composite_risk.csv


,Symbol,Company Name,Prediction Horizon (Years),As Of Date,Predicted Total Return (%),Predicted CAGR (%),Annualized Volatility (%),Latest P/E Ratio,Latest Beta,Composite Risk Score,Risk Band (Composite),Risk Band (Volatility Only),Conservative Band,Expected Band,Optimistic Band
0,GNA.NS,G N A Axles Limited,1,2025-06-18,49.330000,49.330000,37.750000,12.640000,0.210000,0.277500,Low,High,48.33% – 49.33%,50.33% – 52.33%,53.33% – 57.33%
1,CTE.NS,Cambridge Technology Enterprises Limited,1,2025-06-18,49.320000,49.320000,55.110000,nan,-0.420000,nan,Unknown,High,48.32% – 49.32%,50.32% – 52.32%,53.32% – 57.32%
2,AGRITECH.NS,Agri-tech (India) Limited,1,2025-06-18,49.170000,49.170000,58.230000,nan,nan,nan,Unknown,High,48.17% – 49.17%,50.17% – 52.17%,53.17% – 57.17%
3,HINDWAREAP.NS,Hindware Home Innovation Limited,1,2025-06-18,49.160000,49.160000,46.880000,nan,0.580000,nan,Unknown,High,48.16% – 49.16%,50.16% – 52.16%,53.16% – 57.16%
4,KREBSBIO.NS,Krebs Biochemicals & Industries Limited,1,2025-06-18,49.150000,49.150000,67.030000,nan,0.620000,nan,Unknown,High,48.15% – 49.15%,50.15% – 52.15%,53.15% – 57.15%
5,LIKHITHA.NS,Likhitha Infrastructure Limited,1,2025-06-18,48.800000,48.800000,43.960000,16.590000,0.500000,0.353200,Medium,High,47.8% – 48.8%,49.8% – 51.8%,52.8% – 56.8%
6,SHEKHAWATI.NS,Shekhawati Industries Limited,1,2025-06-18,48.670000,48.670000,861.670000,132.830000,1.260000,0.852000,High,High,47.67% – 48.67%,49.67% – 51.67%,52.67% – 56.67%
7,CHEMBOND.NS,Chembond Material Technologies Limited,1,2025-06-18,48.500000,48.500000,53.700000,17.910000,0.610000,0.447100,Medium,High,47.5% – 48.5%,49.5% – 51.5%,52.5% – 56.5%
8,MIRCELECTR.NS,MIRC Electronics Limited,1,2025-06-18,48.410000,48.410000,55.390000,nan,0.200000,nan,Unknown,High,47.41% – 48.41%,49.41% – 51.41%,52.41% – 56.41%
9,DCMSHRIRAM.NS,DCM Shriram Limited,1,2025-06-18,48.240000,48.240000,38.520000,29.870000,-0.220000,0.412600,Medium,High,47.24% – 48.24%,49.24% – 51.24%,52.24% – 56.24%



❌ Preprocessing Stage Failures (sample):
LAXMIDENTL → Insufficient initial data after date filtering (102 rows, 756 required) up to 2025-06-18.
NAVKARURB → Insufficient initial data after date filtering (103 rows, 756 required) up to 2025-06-18.
DDEVPLSTIK → Insufficient initial data after date filtering (105 rows, 756 required) up to 2025-06-18.
QUADFUTURE → Insufficient initial data after date filtering (106 rows, 756 required) up to 2025-06-18.
SGLTL → Insufficient initial data after date filtering (107 rows, 756 required) up to 2025-06-18.
ALIVUS → Insufficient initial data after date filtering (107 rows, 756 required) up to 2025-06-18.
SCODATUBES → Insufficient initial data after date filtering (11 rows, 756 required) up to 2025-06-18.
INDOFARM → Insufficient initial data after date filtering (111 rows, 756 required) up to 2025-06-18.
UNIMECH → Insufficient initial data after date filtering (116 rows, 756 required) up to 2025-06-18.
SENORES → Insufficient initial data after date 

***For Mutual Fund***

In [ ]:
import os
import pandas as pd
import numpy as np
#import numpy_financial as npf
from tqdm import tqdm
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

import scipy.optimize

def my_xirr(values, dates):
    """
    A manual implementation of XIRR to bypass library issues.
    """
    if not all(isinstance(d, pd.Timestamp) for d in dates):
        dates = [pd.to_datetime(d) for d in dates]

    if len(values) != len(dates):
        raise ValueError("values and dates must have the same length")

    if abs(sum(values)) < 1e-6:
        return 0.0

    # Find the time difference in years from the first date
    years = np.array([(d - dates[0]).days / 365.0 for d in dates])

    # Net Present Value (NPV) function
    def npv(rate):
        return np.sum(values / ((1 + rate) ** years))

    try:
        # Use a numerical solver to find the rate where NPV is zero
        return scipy.optimize.newton(npv, 0.1) # 0.1 is an initial guess
    except RuntimeError:
        # If the solver fails to converge, try another guess
        try:
            return scipy.optimize.newton(npv, -0.1)
        except RuntimeError:
            return 0.0 # Return 0 if it still fails

# === CONFIG ===
FOLDER_PATH = "/content/drive/MyDrive/Mutual Funds Data/Output folder"
OUTPUT_CSV = "mutual_fund_predictions_filtered.csv" # Changed output filename
MIN_YEARS_REQUIRED = 3
MIN_OBS = 252 * MIN_YEARS_REQUIRED

# === Result collector
results = []

# === Loop through all files
for file in tqdm(os.listdir(FOLDER_PATH)):
    if not file.endswith(".xlsx"):
        continue

    fund_name = 'Unknown'
    try:
        path = os.path.join(FOLDER_PATH, file)

        # === Read NAV
        nav_df = pd.read_excel(path, sheet_name="NAV")
        nav_df.columns = [col.strip().lower() for col in nav_df.columns]

        if not any("date" in col for col in nav_df.columns) or not any("nav" in col for col in nav_df.columns):
            raise ValueError("Required 'Date' or 'NAV' column not found")

        nav_df.rename(columns={col: 'Date' for col in nav_df.columns if 'date' in col}, inplace=True)
        nav_df.rename(columns={col: 'NAV' for col in nav_df.columns if 'nav' in col}, inplace=True)

        nav_df.dropna(subset=['Date', 'NAV'], inplace=True)
        nav_df['Date'] = pd.to_datetime(nav_df['Date'])
        nav_df.sort_values('Date', inplace=True)
        nav_df.drop_duplicates(subset='Date', keep='first', inplace=True)

        if nav_df.shape[0] < MIN_OBS:
            raise ValueError(f"Insufficient NAV history (<{MIN_YEARS_REQUIRED} years)")

        # === Feature Engineering
        nav_df['Return_1w'] = nav_df['NAV'].pct_change(5)
        nav_df['Return_1m'] = nav_df['NAV'].pct_change(21)
        nav_df['Return_6m'] = nav_df['NAV'].pct_change(126)
        nav_df['Volatility_21d'] = nav_df['NAV'].rolling(21).std()
        nav_df.dropna(inplace=True)

        # === Read Metadata
        meta_df = pd.read_excel(path, sheet_name='Metadata')
        non_numeric_metadata_df = meta_df.select_dtypes(exclude=[np.number])
        numeric_metadata_df = meta_df.select_dtypes(include=[np.number])

        if non_numeric_metadata_df.empty or numeric_metadata_df.empty:
            raise ValueError("Metadata missing numeric/non-numeric rows")

        non_numeric_metadata = non_numeric_metadata_df.iloc[0].to_dict()
        fund_name = non_numeric_metadata.get('fund_name', 'Unknown')
        isin = non_numeric_metadata.get('isin', 'NA')
        category = non_numeric_metadata.get('category', 'NA')
        fund_category = non_numeric_metadata.get('fund_category', 'NA')
        amc = non_numeric_metadata.get('amc', 'NA')

        # === Prepare Model Input
        feature_cols = ['Return_1w', 'Return_1m', 'Return_6m', 'Volatility_21d']
        X = nav_df[feature_cols]
        y = nav_df['NAV']

        model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
        model.fit(X, y)
        preds = model.predict(X)
        rmse = np.sqrt(mean_squared_error(y, preds))
        rmse_percent = (rmse / y.mean()) * 100

        # === Calculate years of NAV data
        total_days = (nav_df['Date'].iloc[-1] - nav_df['Date'].iloc[0]).days
        years_of_data = round(total_days / 365.25, 2)

        # === Lump-Sum CAGR
        cagr_start_date = nav_df['Date'].iloc[-1] - pd.DateOffset(years=MIN_YEARS_REQUIRED)
        cagr_df = nav_df[nav_df['Date'] >= cagr_start_date]

        if len(cagr_df) < 2:
            raise ValueError(f"Not enough data for {MIN_YEARS_REQUIRED}-year CAGR")

        initial_nav = cagr_df['NAV'].iloc[0]
        final_nav = cagr_df['NAV'].iloc[-1]
        cagr = ((final_nav / initial_nav) ** (1 / MIN_YEARS_REQUIRED)) - 1

        # === SIP CAGR (using XIRR)
        sip_cagr = np.nan
        sip_start_date = nav_df['Date'].iloc[-1] - pd.DateOffset(years=MIN_YEARS_REQUIRED)
        monthly_investments = nav_df[nav_df['Date'] >= sip_start_date].set_index('Date').resample('MS').first().dropna()

        if not monthly_investments.empty:
            amount_per_month = 1000
            dates = monthly_investments.index.to_list()
            cash_flows = [-amount_per_month] * len(dates)

            units_purchased = amount_per_month / monthly_investments['NAV']
            final_value = units_purchased.sum() * final_nav

            dates.append(cagr_df['Date'].iloc[-1])
            cash_flows.append(final_value)

            try:
                sip_cagr = my_xirr(cash_flows, dates)
            except Exception as e:
                print(f"      - XIRR calculation failed for {fund_name}: {e}")
                sip_cagr = 0.0

        # === Calculate Annualized Volatility & Risk
        daily_returns = nav_df['NAV'].pct_change()
        annualized_volatility = daily_returns.std() * np.sqrt(252) * 100
        if annualized_volatility < 15:
            risk_band = "Low"
        elif annualized_volatility < 25:
            risk_band = "Medium"
        else:
            risk_band = "High"

        # ==========================================================
        # === NEW: FILTER OUT EXTREME VALUES BEFORE APPENDING
        # ==========================================================
        # Note: We check the raw decimal value (e.g., 0.5 for 50%)

        # Condition 1: Skip if Lump-Sum CAGR is > 50%
        if cagr > 0.50:
            print(f"⚠️ Skipping '{fund_name}' due to high CAGR (>50%): {round(cagr * 100, 2)}%")
            continue # This skips adding the fund to results and moves to the next file

        # Condition 2: Skip if SIP CAGR is > 40%
        if not np.isnan(sip_cagr) and sip_cagr > 0.40:
            print(f"⚠️ Skipping '{fund_name}' due to high SIP CAGR (>40%): {round(sip_cagr * 100, 2)}%")
            continue # This skips adding the fund to results and moves to the next file

        # === Append Result (only if the fund passes the checks above)
        results.append({
            'file': file,
            'isin': isin,
            'fund_name': fund_name,
            'amc': amc,
            'category': category,
            'fund_category': fund_category,
            'years_of_prediction': years_of_data,
            'rmse': round(rmse, 4),
            'rmse_percent': round(rmse_percent, 2),
            'cagr': round(cagr * 100, 2),
            'sip_cagr': round(sip_cagr * 100, 2) if not np.isnan(sip_cagr) else np.nan,
            'annualized_volatility': round(annualized_volatility, 2),
            'risk_band': risk_band,
        })

    except Exception as e:
        print(f"❌ Error processing {file} ({fund_name}): {e}")
        continue

# === Save
pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
print(f"\n✅ Done! {len(results)} funds processed and filtered. Results saved to: {OUTPUT_CSV}")

  0%|          | 2/4144 [00:00<30:09,  2.29it/s]

❌ Error processing QuantPSUIDCWReinvestDirectPlan_INF966L01DV0.xlsx (Unknown): Insufficient NAV history (<3 years)
❌ Error processing SamcoMultiAssetAllocationGrowthDirectPlan_INF0K1H01230.xlsx (Unknown): Insufficient NAV history (<3 years)


  0%|          | 11/4144 [00:02<10:33,  6.52it/s]

❌ Error processing QuantConsumptionGrowthDirectPlan_INF966L01DN7.xlsx (Unknown): Insufficient NAV history (<3 years)


  0%|          | 17/4144 [00:05<27:47,  2.47it/s]

...

❌ Error processing AdityaBirlaSunLifeNiftyIndiaDefenceIndexIDCWPayoutDirectPlan_INF209KC1191.xlsx (Unknown): Insufficient NAV history (<3 years)


100%|█████████▉| 4137/4144 [13:27<00:01,  5.45it/s]

❌ Error processing AxisCrisilIbxAaaBondNbfcJun2027GrowthDirectPlan_INF846K017X1.xlsx (Unknown): Insufficient NAV history (<3 years)


100%|██████████| 4144/4144 [13:28<00:00,  5.13it/s]


❌ Error processing AdityaBirlaSunLifeUSTreasury310YearBondETFsFoFGrowthDirectPlan_INF209KB12X4.xlsx (Unknown): Insufficient NAV history (<3 years)

✅ Done! 3104 funds processed and filtered. Results saved to: mutual_fund_predictions_filtered.csv
